<a href="https://colab.research.google.com/github/ana-rlopez/ny_taxi_fare_prediction/blob/master/NYTaxiFare_1_ExploratoryAnalysis_%26_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from geopy.distance import vincenty
import sklearn.feature_selection
from sklearn import linear_model
sklearn.preprocessing import RobustScaler

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')
#train_path = '/content/drive/My Drive/Colab Notebooks/train.csv'
train_path = 'train.csv'

In [3]:
train = pd.read_csv(train_path, nrows=1_000_000) #we don't select all rows, since it is a big dataset
train.head()

NameError: name 'pd' is not defined

In [ ]:
#check if there are some NaN values
train.columns[train.isna().any()].tolist()

In [ ]:
#we drop these values given that the data set is quite big
#original rows: 1_000_000
train.dropna(inplace=True)
train.shape[0]

In [ ]:
#drop duplicates
train.drop_duplicates(keep = 'first', inplace = True)
train.shape[0]

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
train['key'] = pd.to_datetime(train['key'])
train.dtypes

In [ ]:
#add features related with datetime
train['weekday'] = train['key'].dt.dayofweek

train['year']=train['key'].dt.year
train['month']=train['key'].dt.month
train['day']=train['key'].dt.day
train['hour']=train['key'].dt.hour
#train['minute']=train['key'].dt.minute #needed?
#train['second']=train['key'].dt.second

In [ ]:
train.describe()
#check the describe to see if their values make sense

In [ ]:
#based on these descriptors, we can at first sight already remove some outliers:
#1) Latitudes range from -90 to 90. Longitudes range from -180 to 180.
#2) passengers in a taxi, up to 6 (icnluding suvs) https://ride.guru/lounge/p/how-many-people-can-ride-in-a-cab 
#leave drives of 0 passengers in case documents are transported?
#3) fare has to be positive value, over 2.50$ (that seems to be the initial charge) https://www1.nyc.gov/site/tlc/passengers/taxi-fare.page

train = train[ (train.pickup_longitude >= -180) & (train.pickup_longitude <= 180) & \
              (train.dropoff_longitude >= -180) & (train.dropoff_longitude <= 180) & \
              (train.pickup_latitude >= -90) & (train.pickup_latitude <= 90) & \
              (train.dropoff_latitude >= -90) & (train.dropoff_latitude <= 90) & \
              (train.passenger_count<= 6) & \
              (train.fare_amount > 2.50) ]


In [ ]:
train.describe()

In [ ]:
#maybe convert time (hours and minutes) to circular, given that then the correlation may be more easily seen
#seconds_in_day = 24*60*60

#train['full_time'] = train.hour*60*60 + train.minute*60 + train.second #time of the day in seconds
#train['sin_time'] = np.sin(2*np.pi*train.full_time/seconds_in_day)
#train['cos_time'] = np.cos(2*np.pi*train.full_time/seconds_in_day)
#train.drop('full_time', axis=1, inplace=True)

#note: checked that these 2 feats are computed ok (proper ranges)

In [ ]:
#Instead of doing Harvestine distance (that assumes Earth is a sphere), here Vincenty distance is used,
#which employs more accurate ellipsoidal models

# (latitude, longitude) reminder
train ['distance'] = train.apply(lambda x: vincenty((x['pickup_latitude'], x['pickup_longitude']), (x['dropoff_latitude'], x['dropoff_longitude'])).meters, axis = 1)

In [ ]:
train['distance'].describe()

In [ ]:
train.describe().loc[['min','max']]

In [ ]:
#there shouldn't be any distance equal to 0:
train = train[ (train.distance > 0)]

In [ ]:
#check distributions of all the features
fig = plt.figure(figsize = (15,20))
ax = fig.gca()
train.hist(ax = ax, bins=50)


In [ ]:
Q1 = train.quantile(0.25)
Q3 = train.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
#automatic outlier removal, to try and get more normal distributions
#https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba?gi=8548f80fce4b
train_out = train.drop(['key','pickup_datetime'],axis=1, inplace=False)

train_out = train_out[~((train_out < (Q1 - 1.5 * IQR)) |(train_out > (Q3 + 1.5 * IQR))).any(axis=1)]

train.shape

In [ ]:
train_out.shape

In [ ]:
#this doesn't help (before or after outlier removal)
#Also did log for all feats instead of outlier removal, but it doesnt work much
#train_out['distance'] = train_out['distance'].transform(lambda x: np.log(x+sys.float_info.epsilon))

In [ ]:
#check distributions of all the features
fig2 = plt.figure(figsize = (15,20))
ax2 = fig2.gca()
train_out.hist(ax = ax2, bins=50)

In [ ]:
corr_pearson = train_out.corr(method='pearson')
mask = np.zeros_like(corr_pearson, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

In [ ]:
fig4, ax4 = plt.subplots(figsize=(16, 10))
sns.heatmap(corr_pearson, mask=mask, cmap='RdBu_r', vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5},vmin=-1)

In [ ]:
#visualizer = FeatureCorrelation( method='mutual_info-classification', feature_names=train_out.columns, sort=True)
X = train_out.drop('fare_amount',axis=1, inplace=False).to_numpy()
Y = train_out.fare_amount.to_numpy()
#lab_enc = sklearn.preprocessing.LabelEncoder()
#Y_encoded = lab_enc.fit_transform(Y)

#mutual information has too expensive computationally to run for all data, so we just take 
X = X[1:100000,:]
Y = Y[1:100000]

feature_MIscores = sklearn.feature_selection.mutual_info_regression(X, Y)

#visualizer.fit(X, Y)        # Fit the data to the visualizer
#visualizer.show()              # Finalize and render the figure

In [ ]:
feature_MIscores.shape

#plot also the fare vs the date (year), and maybe check as hypothesis if there is a positive correlation between year and fare

In [ ]:
X.shape

In [ ]:
feats_labels = train_out.drop('fare_amount',axis=1, inplace=False).columns
#print(feats_labels)
index = np.arange(len(feats_labels))
plt.figure(figsize=(10,10))
plt.barh(index, feature_MIscores)
plt.xlabel('mutual information score', fontsize=20)
plt.yticks(index, feats_labels, fontsize=10, rotation=30)

In [ ]:
#TODO: maybe also plot histograms grouping by day, or by hour, to see what times are more common, and see if the fair is similar for 

In [ ]:
#TODO:maybe a heat map of the fares would be interesting, doing a geomap, maybe use plotly?

In [ ]:
#TODO: maybe employ clustering to get some extra information of the data, like outliers? 

We apply the models. 1) Linear regression

In [ ]:
val = pd.read_csv(train_path, skiprows=[2, 1_000_00], nrows=250_000)
val.head()

In [ ]:
val.shape

In [4]:
val.columns[val.isna().any()].tolist()

NameError: name 'val' is not defined

In [ ]:
#we don't remove NaN observations in the valiation test, because we want to be able to check how good it is 
#with all entries
#we use median rather than mean since it is more robust against outliers
val.dropoff_longitude.fillna(val.dropoff_longitude.median(),inplace=True)
val.dropoff_latitude.fillna(val.dropoff_latitude.median(),inplace=True)

In [ ]:
val.columns[val.isna().any()].tolist()

In [ ]:
#now we add the same features that we did previously:
val['key'] = pd.to_datetime(train['key'])
val['weekday'] = val['key'].dt.dayofweek
val['year']=val['key'].dt.year
val['month']=val['key'].dt.month
val['day']=val['key'].dt.day
val['hour']=val['key'].dt.hour

#drop other
val.drop(['key','pickup_datetime'],axis=1, inplace=True)

In [ ]:
#bounding the wrong coordinates (for distance computation)
val.loc[val['pickup_longitude'] > 180, 'pickup_longitude'] = 180
val.loc[val['pickup_longitude'] < -180, 'pickup_longitude'] = -180
val.loc[val['dropoff_longitude'] > 180, 'dropoff_longitude'] = 180
val.loc[val['dropoff_longitude'] < -180, 'dropoff_longitude'] = -180

val.loc[val['pickup_latitude'] > 90, 'pickup_latitude'] = 90
val.loc[val['pickup_latitude'] < -90, 'pickup_latitude'] = -90
val.loc[val['dropoff_latitude'] > 90, 'dropoff_latitude'] = 90
val.loc[val['dropoff_latitude'] < -90, 'dropoff_latitude'] = -90



In [ ]:
val.describe()

In [ ]:
#Instead of doing Harvestine distance (that assumes Earth is a sphere), here Vincenty distance is used,
#which employs more accurate ellipsoidal models

# (latitude, longitude) reminder
val ['distance'] = val.apply(lambda y: vincenty((y['pickup_latitude'], y['pickup_longitude']), (y['dropoff_latitude'], y['dropoff_longitude'])).meters, axis = 1)

In [ ]:
val.describe()

In [ ]:
#normalize data:
#val_norm=(val-val.mean())/val.std()
train_norm = (train-train.mean())/train.std()

In [ ]:
#val_norm.describe()

In [ ]:
# create linear regression model
linRegr = linear_model.LinearRegression()

# train the model with training data
regr.fit(train_norm.drop(['key','pickup_datetime'],axis=1, inplace=False), diabetes_y_train)